<h1 align="center">Segmenting and Clustering Neighborhoods in Toronto City<h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Read and Explore Dataset

In [4]:
df = pd.read_csv("toronto.csv")

In [5]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626
4,M6A,North York,Lawrence Heights,43.7223,-79.4504


##### Use geopy library to get the latitude and longitude values of Toronto City.

In [23]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


##### Create a map of New York with neighborhoods superimposed on top.

In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Let's get the geographical coordinates of Boroughs with "Toronto" in it's name.

In [29]:
tr_df = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)

In [31]:
tr_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
1,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3783
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3783
4,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756


In [32]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(tr_df['Latitude'], tr_df['Longitude'], tr_df['Borough'], tr_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [33]:
CLIENT_ID = 'JWZWPU4GQ1WBMINFHG4WQIDAFAK0AEUP1KUCZV54AZ5VUQXC' # your Foursquare ID
CLIENT_SECRET = 'MPPMNJFUII4K5WRAQ40CEAYVUWTRDBIJD3KP2MEMF4LQOO4W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JWZWPU4GQ1WBMINFHG4WQIDAFAK0AEUP1KUCZV54AZ5VUQXC
CLIENT_SECRET:MPPMNJFUII4K5WRAQ40CEAYVUWTRDBIJD3KP2MEMF4LQOO4W


In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## 2. Explore Neighbourhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [37]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

toronto_venues = getNearbyVenues(names=tr_df['Neighbourhood'],
                                   latitudes=tr_df['Latitude'],
                                   longitudes=tr_df['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [38]:
print(toronto_venues.shape)
toronto_venues.head()

(3306, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,Harbourfront,43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


#### Let's check how many venues were returned for each neighborhood

In [39]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,58,58,58,58,58,58
Berczy Park,93,93,93,93,93,93
Brockton,38,38,38,38,38,38
Business Reply Mail Processing Centre 969 Eastern,14,14,14,14,14,14
CN Tower,58,58,58,58,58,58
Cabbagetown,46,46,46,46,46,46
Central Bay Street,93,93,93,93,93,93
Chinatown,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 224 uniques categories.


## 3. Analyze Each Neighbourhood

In [41]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

manhattan_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0

In [43]:
toronto_onehot.shape

(3306, 224)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.00000,0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.00,0.000000,0.000000,0.070000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00000,0.000000,0.020000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0

In [45]:
toronto_grouped.shape

(72, 224)

#### Let's print each neighborhood along with the top 5 most common venues

In [46]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2   Steakhouse  0.04
3        Hotel  0.04
4          Bar  0.03


----Bathurst Quay----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.07
2          Restaurant  0.05
3                Café  0.05
4                 Bar  0.05


----Berczy Park----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.05
2        Hotel  0.05
3   Restaurant  0.04
4       Bakery  0.04


----Brockton----
                    venue  freq
0             Coffee Shop  0.08
1  Thrift / Vintage Store  0.05
2                    Café  0.05
3          Breakfast Spot  0.05
4            Dance Studio  0.03


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0           Coffee Shop  0.14
1           Yoga Studio  0.07
2  Gym / Fitness Center  0.07
3            Restaurant  0.07
4        Clothing Store  0.07


----CN Tower----
                venue  freq
0         Coffee Sh

                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.07
2          Restaurant  0.05
3                Café  0.05
4                 Bar  0.05


----Rathnelly----
                venue  freq
0  Light Rail Station  0.29
1         Coffee Shop  0.29
2  Photography Studio  0.14
3        Liquor Store  0.14
4         Supermarket  0.14


----Regent Park----
                  venue  freq
0           Coffee Shop  0.19
1  Gym / Fitness Center  0.07
2            Restaurant  0.07
3        Breakfast Spot  0.07
4           Yoga Studio  0.04


----Richmond----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2   Steakhouse  0.04
3        Hotel  0.04
4          Bar  0.03


----Riverdale----
                venue  freq
0    Greek Restaurant  0.24
1      Ice Cream Shop  0.05
2  Italian Restaurant  0.05
3                Café  0.05
4         Coffee Shop  0.05


----Roncesvalles----
                         venue  freq
0  Eastern European Restaurant  0.07
1           

#### Let's put that into a pandas dataframe

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Café,Coffee Shop,Steakhouse,Hotel,Gastropub,Bar,Thai Restaurant,Restaurant,Burger Joint,Asian Restaurant
1,Bathurst Quay,Coffee Shop,Italian Restaurant,Café,Restaurant,Bar,Park,Bakery,Speakeasy,Pub,Sandwich Place
2,Berczy Park,Coffee Shop,Café,Hotel,Bakery,Restaurant,Beer Bar,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Japanese Restaurant
3,Brockton,Coffee Shop,Breakfast Spot,Café,Thrift / Vintage Store,Diner,Brewery,Sandwich Place,Caribbean Restaurant,Restaurant,Chiropractor
4,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Yoga Studio,Italian Restaurant,Deli / Bodega,Clothing Store,Restaurant,Martial Arts Dojo,Bookstore,Sushi Restaurant,Japanese Restaurant


## 4. Cluster Neighbourhoods

In [64]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tr_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626,1.0,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Restaurant,Bakery,Deli / Bodega,Playground,Pub,Mexican Restaurant,Electronics Store
1,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626,1.0,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Restaurant,Bakery,Deli / Bodega,Playground,Pub,Mexican Restaurant,Electronics Store
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3783,1.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Plaza,Pizza Place,Hotel
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3783,1.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Plaza,Pizza Place,Hotel
4,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,1.0,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Pizza Place,Seafood Restaurant,Breakfast Spot,Cosmetics Shop


In [78]:
toronto_merged.dropna(inplace=True)
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype("int")
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626,1,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Restaurant,Bakery,Deli / Bodega,Playground,Pub,Mexican Restaurant,Electronics Store
1,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626,1,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Restaurant,Bakery,Deli / Bodega,Playground,Pub,Mexican Restaurant,Electronics Store
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3783,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Plaza,Pizza Place,Hotel
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3783,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Plaza,Pizza Place,Hotel
4,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Pizza Place,Seafood Restaurant,Breakfast Spot,Cosmetics Shop


#### Finally, let's visualize the resulting clusters

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Deer Park,0,Light Rail Station,Coffee Shop,Photography Studio,Liquor Store,Supermarket,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
55,Forest Hill SE,0,Light Rail Station,Coffee Shop,Photography Studio,Liquor Store,Supermarket,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
56,Rathnelly,0,Light Rail Station,Coffee Shop,Photography Studio,Liquor Store,Supermarket,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
57,South Hill,0,Light Rail Station,Coffee Shop,Photography Studio,Liquor Store,Supermarket,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
58,Summerhill West,0,Light Rail Station,Coffee Shop,Photography Studio,Liquor Store,Supermarket,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### Cluster 2

In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,1,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Restaurant,Bakery,Deli / Bodega,Playground,Pub,Mexican Restaurant,Electronics Store
1,Regent Park,1,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Restaurant,Bakery,Deli / Bodega,Playground,Pub,Mexican Restaurant,Electronics Store
2,Ryerson,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Plaza,Pizza Place,Hotel
3,Garden District,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Plaza,Pizza Place,Hotel
4,St. James Town,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Pizza Place,Seafood Restaurant,Breakfast Spot,Cosmetics Shop
5,The Beaches,1,Pub,Trail,Bakery,Gastropub,Health Food Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dog Run
6,Berczy Park,1,Coffee Shop,Café,Hotel,Bakery,Restaurant,Beer Bar,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Japanese Restaurant
7,Central Bay Street,1,Coffee Shop,Middle Eastern Restaurant,Tea Room,Café,Clothing Store,Italian Restaurant,Chinese Restaurant,Breakfast Spot,Bubble Tea Shop,Spa
8,Christie,1,Grocery Store,Café,Athletics & Sports,Baby Store,Park,Playground,Coffee Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant
9,Adelaide,1,Café,Coffee Shop,Steakhouse,Hotel,Gastropub,Bar,Thai Restaurant,Restaurant,Burger Joint,Asian Restaurant


### Cluster 3

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,High Park,2,Park,Women's Store,Doner Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
37,The Junction South,2,Park,Women's Store,Doner Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### Cluster 4

In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Forest Hill North,3,Park,Trail,Furniture / Home Store,Dry Cleaner,Women's Store,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
35,Forest Hill West,3,Park,Trail,Furniture / Home Store,Dry Cleaner,Women's Store,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### Cluster 5

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Harbourfront East,4,Park,Music Venue,Harbor / Marina,Café,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
15,Toronto Islands,4,Park,Music Venue,Harbor / Marina,Café,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
16,Union Station,4,Park,Music Venue,Harbor / Marina,Café,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
31,Lawrence Park,4,Photography Studio,Park,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
38,North Toronto West,4,Garden,Gym Pool,Playground,Park,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
49,Moore Park,4,Park,Playground,Gym,Grocery Store,Thai Restaurant,Women's Store,Eastern European Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant
50,Summerhill East,4,Park,Playground,Gym,Grocery Store,Thai Restaurant,Women's Store,Eastern European Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant
66,Rosedale,4,Bank,Park,Playground,Grocery Store,Candy Store,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
